In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\work\\spark-2.4.8-bin-hadoop2.7'

In [2]:
spark.stop()

NameError: name 'spark' is not defined

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("customer-orders").getOrCreate()
# spark=SparkSession.builder.appName("customer-orders").master("spark://DESKTOP-DELTA24.localdomain:7077").getOrCreate()
spark

In [4]:
from pyspark.sql.types import *
schema=StructType([
    StructField("CustomerID",IntegerType(),True),
    StructField("OrderID",IntegerType(),True),
    StructField("Cost",FloatType(),True),
])

In [5]:
data=spark.read.schema(schema).csv("customer-orders.csv")
data.show(5)

+----------+-------+-----+
|CustomerID|OrderID| Cost|
+----------+-------+-----+
|        44|   8602|37.19|
|        35|   5368|65.89|
|         2|   3391|40.64|
|        47|   6694|14.98|
|        29|    680|13.08|
+----------+-------+-----+
only showing top 5 rows



In [8]:
df=data.select("CustomerID","Cost")

In [9]:
from pyspark.sql import functions as func
df.groupBy("CustomerID").agg(func.sum("Cost").alias("Cost")).show(5)

[Stage 1:>                                                          (0 + 1) / 1]

+----------+-----------------+
|CustomerID|             Cost|
+----------+-----------------+
|        31|4765.050008416176|
|        85| 5503.42998456955|
|        65|5140.349995829165|
|        53|4945.300026416779|
|        78|4524.510001778603|
+----------+-----------------+
only showing top 5 rows



## Partitions

In [18]:
data.rdd.getNumPartitions()

1

In [19]:
partition=data.limit(200).repartition(50)
partition.rdd.getNumPartitions()

50

In [20]:
partition.rdd.glom().take(2)

[[Row(CustomerID=78, OrderID=1085, Cost=16.959999084472656),
  Row(CustomerID=86, OrderID=795, Cost=79.83999633789062),
  Row(CustomerID=40, OrderID=3083, Cost=72.94999694824219),
  Row(CustomerID=62, OrderID=5982, Cost=41.77000045776367)],
 [Row(CustomerID=60, OrderID=2870, Cost=31.850000381469727),
  Row(CustomerID=50, OrderID=3901, Cost=23.56999969482422),
  Row(CustomerID=57, OrderID=4760, Cost=37.470001220703125),
  Row(CustomerID=90, OrderID=2324, Cost=87.2300033569336)]]

In [21]:
partition.orderBy("CustomerID","OrderID").show(5000)

+----------+-------+-----+
|CustomerID|OrderID| Cost|
+----------+-------+-----+
|         0|   1593| 59.8|
|         0|   3479|97.22|
|         1|   6725| 9.99|
|         2|   3391|40.64|
|         2|   4326|10.46|
|         2|   4424|55.77|
|         2|   8212|20.31|
|         2|   9150|76.88|
|         4|   5640|84.57|
|         4|   7236|24.66|
|         4|   7577|24.13|
|         5|   3505|64.18|
|         6|    784|51.88|
|         6|   4873|44.47|
|         6|   7808| 42.3|
|         7|   3530|51.38|
|         7|   9541|40.34|
|         8|   2826| 9.68|
|         9|   3182|62.76|
|        10|    987|74.45|
|        10|   3471|68.65|
|        10|   8643|89.53|
|        11|   8220|92.23|
|        12|   4396|72.62|
|        13|   1298| 10.4|
|        13|   4507|26.47|
|        14|   1505| 4.32|
|        14|   5388|77.77|
|        14|   5998|80.53|
|        15|   5033|88.82|
|        15|   5062|56.94|
|        15|   6148|65.53|
|        16|    424|90.22|
|        16|   1415| 15.7|
|

In [22]:
repartition=partition.repartition(5)
repartition=repartition.repartition(20)

In [23]:
repartition.sort("CustomerID","OrderID").show(5000)

+----------+-------+-----+
|CustomerID|OrderID| Cost|
+----------+-------+-----+
|         0|   1593| 59.8|
|         0|   3479|97.22|
|         1|   6725| 9.99|
|         2|   3391|40.64|
|         2|   4326|10.46|
|         2|   4424|55.77|
|         2|   8212|20.31|
|         2|   9150|76.88|
|         4|   5640|84.57|
|         4|   7236|24.66|
|         4|   7577|24.13|
|         5|   3505|64.18|
|         6|    784|51.88|
|         6|   4873|44.47|
|         6|   7808| 42.3|
|         7|   3530|51.38|
|         7|   9541|40.34|
|         8|   2826| 9.68|
|         9|   3182|62.76|
|        10|    987|74.45|
|        10|   3471|68.65|
|        10|   8643|89.53|
|        11|   8220|92.23|
|        12|   4396|72.62|
|        13|   1298| 10.4|
|        13|   4507|26.47|
|        14|   1505| 4.32|
|        14|   5388|77.77|
|        14|   5998|80.53|
|        15|   5033|88.82|
|        15|   5062|56.94|
|        15|   6148|65.53|
|        16|    424|90.22|
|        16|   1415| 15.7|
|

In [24]:
repartition.orderBy("CustomerID","OrderID").show(5000)

+----------+-------+-----+
|CustomerID|OrderID| Cost|
+----------+-------+-----+
|         0|   1593| 59.8|
|         0|   3479|97.22|
|         1|   6725| 9.99|
|         2|   3391|40.64|
|         2|   4326|10.46|
|         2|   4424|55.77|
|         2|   8212|20.31|
|         2|   9150|76.88|
|         4|   5640|84.57|
|         4|   7236|24.66|
|         4|   7577|24.13|
|         5|   3505|64.18|
|         6|    784|51.88|
|         6|   4873|44.47|
|         6|   7808| 42.3|
|         7|   3530|51.38|
|         7|   9541|40.34|
|         8|   2826| 9.68|
|         9|   3182|62.76|
|        10|    987|74.45|
|        10|   3471|68.65|
|        10|   8643|89.53|
|        11|   8220|92.23|
|        12|   4396|72.62|
|        13|   1298| 10.4|
|        13|   4507|26.47|
|        14|   1505| 4.32|
|        14|   5388|77.77|
|        14|   5998|80.53|
|        15|   5033|88.82|
|        15|   5062|56.94|
|        15|   6148|65.53|
|        16|    424|90.22|
|        16|   1415| 15.7|
|